In [2]:
import random
import math
import time
from concurrent.futures import ThreadPoolExecutor

def get_neighbors(x, k):
    n = []
    if x == 1:
        n = [2 * k, x + 1]
    elif x == 2 * k:
        n = [x - 1, 1]
    else:
        n = [x - 1, x + 1]

    if x > k:
        n.append(x - k)
    else:
        n.append(x + k)
    return n

def run_game_once(k):
    nodes = 2 * k
    M = {i + 1: get_neighbors(i + 1, k) for i in range(nodes)}

    c = random.randint(1, nodes)
    r_candidates = [i for i in M.keys() if not i == c and not i in M[c]]
    r = random.choice(r_candidates)

    r_move_dir = ""
    c_move_dir = ""

    damaged = set()

    for i in range(100000):
        if i % 2 == 0:
            c_n = M[c]
            r_n = M[r]

            common_n = list(set(c_n) & set(r_n))

            if len(common_n) > 0:
                pass
            else:
                if r_move_dir == "b" or r_move_dir == "f":
                    c = c_n[0] if r_move_dir == "f" else c_n[1]
                else:
                    if c_move_dir == "f":
                        c = c_n[0]
                        c_move_dir = "b"
                    elif c_move_dir == "b":
                        c = c_n[1]
                        c_move_dir = "f"
                    else:
                        c = c_n[0]
                        c_move_dir = "b"
        else:
            c_n = M[c]
            r_n = M[r]
            common_n = list(set(c_n) & set(r_n))
            valid_moves = [i for i in r_n if i not in common_n]
            r1 = random.choice(valid_moves)
            damaged.add(r)
            r = r1
            r_move_dir = "b" if r == r_n[0] else "f" if r == r_n[1] else "a"

    return damaged

def run_game_n_times_parallel(n, k):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda _: run_game_once(k), range(n)))

    return results

def generate_damage_table_parallel():
    results = []
    for k_value in range(5, 25, 2):
        n = 1
        damaged_nodes_list = run_game_n_times_parallel(n, k_value)
        total_damaged_nodes = sum(len(nodes) for nodes in damaged_nodes_list)
        results.append((k_value, total_damaged_nodes, damaged_nodes_list))

    return results

if __name__ == "__main__":
    start_time = time.time()
    table = generate_damage_table_parallel()
    end_time = time.time()

    # Print the table
    print("\nk\tTotal Damaged Nodes\tDamaged Nodes")
    print("----------------------------------------------")
    for k, total_damaged_nodes, damaged_nodes_list in table:
        print(f"{k}\t{total_damaged_nodes}\t\t\t{damaged_nodes_list}")

    # Print the total time taken
    print(f"\nTotal Time Taken: {end_time - start_time:.6f} seconds")




k	Total Damaged Nodes	Damaged Nodes
----------------------------------------------
5	4			[{1, 10, 5, 6}]
7	6			[{3, 4, 5, 10, 11, 12}]
9	8			[{2, 3, 4, 5, 10, 11, 12, 13}]
11	10			[{1, 2, 3, 4, 10, 11, 12, 13, 14, 22}]
13	12			[{1, 2, 3, 12, 13, 14, 15, 16, 17, 24, 25, 26}]
15	14			[{1, 11, 12, 13, 14, 15, 16, 17, 25, 26, 27, 28, 29, 30}]
17	16			[{1, 2, 34, 33, 3, 4, 5, 6, 16, 17, 18, 19, 20, 21, 22, 23}]
19	18			[{32, 33, 6, 7, 8, 9, 10, 11, 12, 13, 14, 25, 26, 27, 28, 29, 30, 31}]
21	20			[{9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39}]
23	22			[{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37}]

Total Time Taken: 2.738617 seconds
